# Introduction to Geospatial AI

# Intro
Welcome to this workship about geospatial AI! In this workshop you will try to detect buildings from aerial images. This is done in three steps;

1. Creating training data.
2. Training machine learning models.
3. Evaluating the trained models and predicting where buildings are in images the models haven't seen before. 

We will be using jupyter notebooks with Google Colab, but you don't need to have any experience with these in order to complete this workshop.

# Task 0

But first, before we can do any of the fun stuff, we need to set up the environment properly. In order to do that follow the steps under;

1. Create a Google account, for instance by creating a gmail account. Can be skipped if you already have a gmail account. Log into the account in your browser.
2. Head over to https://colab.research.google.com/ and press on the `Github` tab. Search for `kartAI` user and select `kartAI/kartAI` repository. A notebook should appear. Press on this notebook and it should open a notebook in another tab.
3. Create a copy of the notebook in your drive by saving it. Shortcut is `Ctrl + s` on Windows or `Cmd + s` on Mac.
4. Change to a GPU runtime environment. In the top right corner choose `Change runtime type` and select `T4 GPU`.
5. Insert secrets. # TODO: Mayhaps

Nice! What's left now is to clone the git repo we are working with and adding it to the path in addition to installing some dependencies. To do this simply run the two cells below.

In [ ]:
!git clone https://github.com/kartAI/kartAI.git

!pip install focal_loss
!pip install azure-storage-blob
!pip install rasterio
!pip install rasterstats

In [ ]:
import sys
sys.path.insert(0,'/content/kartAI')

# Task 1 - Create training data
In the first task we create the training data. This is done by selecting an area you want to train on. We then download aerial photos for that area in addition to data about all the existing 

https://geojson.io/

In [ ]:
from kartAI.kartai.tools.create_training_data import create_training_data

create_training_data(training_dataset_name="test", config_file_path="kartAI/config/dataset/bygg.json", eager_load=True,
                         confidence_threshold=None, eval_model_checkpoint=None,
                         region=None, x_min=618296.0, x_max=621495.0, y_min=6668145.0, y_max=6670133.0,
                         num_processes=None)

In [ ]:
import folium
import rasterio
import os

from pyproj import CRS
from pyproj import Transformer

path_to_dir = "/content/training_data/OrtofotoWMS/25832_563000.0_6623000.0_100.0_100.0/512/"
files = os.listdir(path_to_dir)
files.sort()

crs_25832 = CRS.from_epsg(25832)
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(crs_25832, crs_4326)

fig = folium.Figure(width=800, height=400)
m = folium.Map(location=transformer.transform(618200.0, 6669700), zoom_start=14)

for i in range(5):
    with rasterio.open(f"{path_to_dir}{files[i]}") as src:
        img = src.read()
        transformed_bottom_left = transformer.transform(src.bounds.left, src.bounds.bottom)
        transformed_top_right = transformer.transform(src.bounds.right, src.bounds.top)
    m.add_child(folium.raster_layers.ImageOverlay(img.transpose(1, 2, 0), bounds = [transformed_bottom_left, transformed_top_right]))

fig.add_child(m)

# Task 2 - Train a machine learning model

In [ ]:
from kartAI.kartai.tools.train import train

train_args = {
        "features": 8,
        "depth": 1,
        "optimizer": "RMSprop",
        "batch_size": 8,
        "model": "unet",
        "loss": "binary_crossentropy",
        "activation": "relu",
        "epochs": 1
}


train(checkpoint_name="some_checkpoint",
      dataset_name=["test"],
      input_generator_config_path="kartAI/config/ml_input_generator/ortofoto.json",
      save_model=False,
      train_args=train_args,
      checkpoint_to_finetune=False
)


# Task 3 - Evaluation and Inference

In [ ]:
import os
import json
from kartAI.env import get_env_variable
from kartAI.kartai.tools.predict import predict_and_evaluate

created_datasets_dir = os.path.join(get_env_variable(
    'created_datasets_directory'), "test")

checkpoint_path = os.path.join(get_env_variable(
    'trained_models_directory'), 'some_checkpoint.h5')

with open("kartAI/config/ml_input_generator/ortofoto.json", encoding="utf8") as config:
    datagenerator_config = json.load(config)

predict_and_evaluate(
    created_datasets_dir,
    datagenerator_config,
    "some_checkpoint",
    True,
    True
)

In [ ]:
from kartAI.kartai.dataset.create_building_dataset import produce_vector_buildings, run_ml_predictions
from kartAI.kartai.utils.config_utils import read_config
from kartAI.kartai.utils.crs_utils import get_projection_from_config_path
from kartAI.kartai.utils.geometry_utils import parse_region_arg
from kartAI.kartai.utils.prediction_utils import get_raster_predictions_dir, get_vector_predictions_dir
from kartAI.kartai.utils.train_utils import get_existing_model_names

geom = parse_region_arg("kartAI/training_data/regions/small_test_region.json")

projection = get_projection_from_config_path("kartAI/config/dataset/bygg.json")

config = read_config("kartAI/config/dataset/bygg.json")

run_ml_predictions("some_checkpoint", "small_test_region", projection,
                    config=config, geom=geom, batch_size=200, skip_data_fetching=False,
                    save_to="local", num_processes=1)

vector_output_dir = get_vector_predictions_dir(
    "small_test_region", "some_checkpoint")
raster_predictions_path = get_raster_predictions_dir(
    "small_test_region", "some_checkpoint")

produce_vector_buildings(
    vector_output_dir, raster_predictions_path, config, 200, "small_test_region_some_checkpoint", save_to="local")

In [ ]:
# Polygon
import folium
import geopandas as gp

data_25832 = gp.read_file("results/small_test_region/some_checkpoint/vector/raw_predictions_0.json")
data_4326 = data_25832.to_crs(4326)

print(data_4326.crs)
print(data_4326)

fig = folium.Figure(width=800, height=400)
map = folium.Map(location=transformer.transform(618200.0, 6669700), zoom_start=14)
folium.GeoJson(data=data_4326["geometry"]).add_to(map)
fig.add_child(map)

In [ ]:
from kartAI.kartai.dataset.create_building_dataset import run_ml_predictions
from kartAI.kartai.tools.predict import create_contour_result
from kartAI.kartai.utils.config_utils import read_config
from kartAI.kartai.utils.crs_utils import get_projection_from_config_path
from kartAI.kartai.utils.geometry_utils import parse_region_arg
from kartAI.kartai.utils.prediction_utils import get_contour_predictions_dir, get_raster_predictions_dir
from kartAI.kartai.utils.train_utils import get_existing_model_names

geom = parse_region_arg("kartAI/training_data/regions/small_test_region.json")

projection = get_projection_from_config_path("kartAI/config/dataset/bygg.json")

config = read_config("kartAI/config/dataset/bygg.json")

run_ml_predictions("some_checkpoint", "small_test_region", projection,
                    config=config, geom=geom, batch_size=200, skip_data_fetching=False,
                    save_to="local", num_processes=1)

raster_output_dir = get_raster_predictions_dir(
    "small_test_region", "some_checkpoint")
contour_output_dir = get_contour_predictions_dir(
    "small_test_region", "some_checkpoint")

print("---> Creating contour dataset from rasters")
contour_levels = [0.3, 0.4, 0.5, 0.6, 0.8, 0.9, 1]
create_contour_result(
    raster_output_dir, contour_output_dir, projection, contour_levels)

print("==== Contour dataset created ====")


In [ ]:
# Contour
import folium
import geopandas as gp

data_25832 = gp.read_file("results/small_test_region/some_checkpoint/contour/complete_contour.json")
data_4326 = data_25832.to_crs(4326)

print(data_4326.crs)
print(data_4326)

fig = folium.Figure(width=800, height=400)
map = folium.Map(location=transformer.transform(618200.0, 6669700), zoom_start=14)
folium.GeoJson(data=data_4326["geometry"]).add_to(map)
fig.add_child(map)